<a href="https://colab.research.google.com/github/hasiburrahman1/NLP/blob/master/method_2_Embedding_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import tensorflow as tf

In [ ]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
tf.__version__ 

'2.2.0'

In [ ]:
all_data = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/train.tsv",sep='\t',encoding = "ISO-8859-1")

### temporary 

In [ ]:
from sklearn.model_selection import train_test_split
data_raw, y_fifty_pecsent = train_test_split(all_data, test_size = 0.70, random_state = 0)

print("Number of rows in data =",data_raw.shape[0])
print("Number of columns in data =",data_raw.shape[1])

Number of rows in data = 13479
Number of columns in data = 3


In [ ]:
data_raw.head()

,label,question,answer_candidate
3300,0,Do R-loops tend to form at sites of DNA replic...,Mitochondrial DNA_replication at the leading-s...
20074,0,Which miRNAs could be used as potential biomar...,Our results_indicate that miRNAs might play a ...
30894,0,What is the role of the Tsix gene during X chr...,"Here , we show that reduction of DICER1 in hum..."
17616,0,Which are the major characteristics of cellula...,In addition microglia seems to develop charact...
20331,0,Is phospholamban a regulatory/inhibitory prote...,"Herein , we provide a detailed protocol for th..."


##### Preprocessing 

In [ ]:
# Removing punctuations
data=data_raw.iloc[:,1:3]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# Renaming column names for ease of access
list1= [i for i in range(2)]
new_Index=[str(i) for i in list1]
data.columns= new_Index
data.head(5)

,0,1
3300,Do R loops tend to form at sites of DNA replic...,Mitochondrial DNA replication at the leading s...
20074,Which miRNAs could be used as potential biomar...,Our results indicate that miRNAs might play a ...
30894,What is the role of the Tsix gene during X chr...,Here we show that reduction of DICER in hum...
17616,Which are the major characteristics of cellula...,In addition microglia seems to develop charact...
20331,Is phospholamban a regulatory inhibitory prote...,Herein we provide a detailed protocol for th...


In [ ]:
' '.join(str(x) for x in data.iloc[0,0:3])

'Do R loops tend to form at sites of DNA replication  Mitochondrial DNA replication at the leading strand origin is coupled to transcription through the formation of an RNA DNA hybrid known as an R loop'

In [ ]:
# Convertng headlines to lower case
for index in new_Index:
    data[index]=data[index].str.lower()
data.head(5)    

,0,1
3300,do r loops tend to form at sites of dna replic...,mitochondrial dna replication at the leading s...
20074,which mirnas could be used as potential biomar...,our results indicate that mirnas might play a ...
30894,what is the role of the tsix gene during x chr...,here we show that reduction of dicer in hum...
17616,which are the major characteristics of cellula...,in addition microglia seems to develop charact...
20331,is phospholamban a regulatory inhibitory prote...,herein we provide a detailed protocol for th...


In [ ]:
' '.join(str(x) for x in data.iloc[0,0:3])

'do r loops tend to form at sites of dna replication  mitochondrial dna replication at the leading strand origin is coupled to transcription through the formation of an rna dna hybrid known as an r loop'

In [ ]:
corpus = []
for row in range(0,len(data.index)):
    corpus.append(' '.join(str(x) for x in data.iloc[row,0:3]))

In [ ]:
corpus[0]

'do r loops tend to form at sites of dna replication  mitochondrial dna replication at the leading strand origin is coupled to transcription through the formation of an rna dna hybrid known as an r loop'

In [ ]:
corpus

['do r loops tend to form at sites of dna replication  mitochondrial dna replication at the leading strand origin is coupled to transcription through the formation of an rna dna hybrid known as an r loop',
 'which mirnas could be used as potential biomarkers for epithelial ovarian cancer  our results indicate that mirnas might play a role in the pathogenesis of human eoc and identify altered mirna gene methylation as a possible epigenetic mechanism involved in their aberrant expression',
 'what is the role of the tsix gene during x chromosome inactivation  here   we show that reduction of dicer  in human female cells increases xist transcripts without compromising the binding of the xist and histone tail modifications on the xi  ',
 'which are the major characteristics of cellular senescence  in addition microglia seems to develop characteristics that could be related to cellular senescence post hiv   infection and after exposure to hiv   viral proteins',
 'is phospholamban a regulator

#### Implement BAG OF WORDS

In [ ]:
'''
from sklearn.feature_extraction.text import CountVectorizer

countvector=CountVectorizer(ngram_range=(2,2))
X=countvector.fit_transform(corpus)
'''

'\nfrom sklearn.feature_extraction.text import CountVectorizer\n\ncountvector=CountVectorizer(ngram_range=(2,2))\nX=countvector.fit_transform(corpus)\n'

#### TFidf Vectorizer

In [ ]:
'''
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_v=TfidfVectorizer(max_features=10000,ngram_range=(1,3))
X=tfidf_v.fit_transform(corpus).toarray()
'''

'\nfrom sklearn.feature_extraction.text import TfidfVectorizer\n\ntfidf_v=TfidfVectorizer(max_features=10000,ngram_range=(1,3))\nX=tfidf_v.fit_transform(corpus).toarray()\n'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

#### One hot representation

In [ ]:
### Vocabulary size
voc_size=10000

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[5861,
  6268,
  7531,
  6272,
  8886,
  8436,
  2595,
  8052,
  3262,
  1967,
  4092,
  4917,
  1967,
  4092,
  2595,
  8138,
  8839,
  3999,
  1567,
  3906,
  8237,
  8886,
  9882,
  5801,
  8138,
  2918,
  3262,
  2983,
  1470,
  1967,
  5026,
  5643,
  7355,
  2983,
  6268,
  9565],
 [6557,
  5847,
  7396,
  4290,
  4622,
  7355,
  4985,
  1203,
  3558,
  5430,
  6184,
  847,
  8020,
  4392,
  5459,
  5637,
  5847,
  2992,
  3188,
  6985,
  5490,
  3527,
  8138,
  2759,
  3262,
  8534,
  358,
  5729,
  6690,
  5978,
  2455,
  663,
  234,
  7355,
  6985,
  5260,
  6069,
  8385,
  6248,
  3527,
  7625,
  5480,
  6546],
 [8375,
  3906,
  8138,
  5490,
  3262,
  8138,
  2164,
  663,
  5561,
  6508,
  747,
  5425,
  1123,
  998,
  9525,
  5637,
  4512,
  3262,
  6989,
  3527,
  8534,
  2635,
  3893,
  3706,
  1517,
  5452,
  140,
  8679,
  8138,
  6626,
  3262,
  8138,
  1517,
  5729,
  556,
  9036,
  9380,
  9730,
  8138,
  9760],
 [6557,
  742,
  8138,
  2413,
  2464,
  3262,
  1461,

#### Embedding Representation

In [ ]:
sent_length = len(max(corpus, key=len))
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2983 6268 9565]
 [   0    0    0 ... 7625 5480 6546]
 [   0    0    0 ... 9730 8138 9760]
 ...
 [   0    0    0 ... 3527 7421 9322]
 [   0    0    0 ... 3527 8138 4976]
 [   0    0    0 ...  448 9696 8199]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0, ..., 2983, 6268, 9565], dtype=int32)

### LSTM model

In [ ]:
## Creating model

embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3521, 40)          400000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
## Creating Bidirectional LSTM model

embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3521, 40)          400000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 513,001
Trainable params: 513,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
## Creating LSTM with dropout ratio model

embedding_vector_features=40
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model2.add(Dropout(0.3))
model2.add(LSTM(100))
model2.add(Dropout(0.3))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3521, 40)          400000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 513,001
Trainable params: 513,001
Non-trainable params: 0
_________________________________________________________________
None


##### Get the Independent and Dependent Features

In [ ]:
len(embedded_docs)

13479

In [ ]:
y=pd.get_dummies(data_raw['label'])
y=y.iloc[:,1].values

In [ ]:
X=np.array(embedded_docs)
y=np.array(y)

###### Train Test Split from sklearn

##### LSTM Model Training

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [ ]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X,y),epochs=10,batch_size=64)

Epoch 1/10
142/142 [==============================] - 76s 534ms/step - loss: 0.1125 - accuracy: 0.9852 - val_loss: 0.0695 - val_accuracy: 0.9871
Epoch 2/10
142/142 [==============================] - 75s 531ms/step - loss: 0.0757 - accuracy: 0.9858 - val_loss: 0.0679 - val_accuracy: 0.9871
Epoch 3/10
142/142 [==============================] - 74s 523ms/step - loss: 0.0650 - accuracy: 0.9858 - val_loss: 0.0478 - val_accuracy: 0.9871
Epoch 4/10
142/142 [==============================] - 73s 512ms/step - loss: 0.0291 - accuracy: 0.9893 - val_loss: 0.0334 - val_accuracy: 0.9941
Epoch 5/10
142/142 [==============================] - 73s 512ms/step - loss: 0.0102 - accuracy: 0.9976 - val_loss: 0.0350 - val_accuracy: 0.9950
Epoch 6/10
142/142 [==============================] - 73s 512ms/step - loss: 0.0052 - accuracy: 0.9984 - val_loss: 0.0346 - val_accuracy: 0.9955
Epoch 7/10
142/142 [==============================] - 73s 511ms/step - loss: 0.0042 - accuracy: 0.9990 - val_loss: 0.0348 - val_ac

##### Performance Metrics And Accuracy of LSTM model

In [ ]:
y_pred1= np.argmax(model1.predict(X_test), axis=-1) 

In [ ]:
## Import library to check accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score

In [ ]:
matrix=confusion_matrix(y_test,y_predict)
print(matrix)

[[4403    0]
 [  44    2]]


In [ ]:
accuracy_score(y_test,y_pred1)

0.9896605978871656

In [ ]:
report=classification_report(y_test,y_pred1)
print(report)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      4403
           1       0.00      0.00      0.00        46

    accuracy                           0.99      4449
   macro avg       0.49      0.50      0.50      4449
weighted avg       0.98      0.99      0.98      4449



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### RandomForest Classifier

In [ ]:
## implement RandomForest Classifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(X_train,y_train)

y_predict = randomclassifier.predict(X_test)

In [ ]:
matrix=confusion_matrix(y_test,y_predict)
print(matrix)
score=accuracy_score(y_test,y_predict)
print(score)

[[4403    0]
 [  44    2]]
0.9901101371094628


In [ ]:
report=classification_report(y_test,y_predict)
print(report)

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      4403
           1       1.00      0.04      0.08        46

    accuracy                           0.99      4449
   macro avg       1.00      0.52      0.54      4449
weighted avg       0.99      0.99      0.99      4449

